<a href="https://colab.research.google.com/github/erlichsefi/ScrapeAnything/blob/main/browser_base_translation%20/AutoJavaScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoGPT with JavaScript

In [22]:
import dotenv
# to get the openai key
assert dotenv.load_dotenv("/workspaces/ScrapeAnything/.env", override=True), "please create .env file"

## install selenium & chromium

In [23]:
# # According to: https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com
# %%capture
# %%shell
# # Ubuntu no longer distributes chromium-browser outside of snap
# #
# # Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# # Add debian buster
# cat > /etc/apt/sources.list.d/debian.list <<'EOF'
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
# EOF

# # Add keys
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

# apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
# apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
# apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# # Prefer debian repo for chromium* packages only
# # Note the double-blank lines between entries
# cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
# Package: *
# Pin: release a=eoan
# Pin-Priority: 500


# Package: *
# Pin: origin "deb.debian.org"
# Pin-Priority: 300


# Package: chromium*
# Pin: origin "deb.debian.org"
# Pin-Priority: 700
# EOF

# # Install chromium and chromium-driver
# apt-get update
# apt-get install chromium chromium-driver

# # Install selenium
# pip install selenium
# apt install chromium-chromedriver
# pip install pandas
# pip install openai
# pip install Pillow
# pip install pydantic

## util to convert the on screen to data

In [24]:
def start_browesr():
  from selenium import webdriver
  from selenium.webdriver.chrome.service import Service

  
  chrome_options = webdriver.ChromeOptions()
  #chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--lang=en')
  #chrome_options.headless = True
  return webdriver.Remote("http://host.docker.internal:4444/wd/hub",options=chrome_options)
  #service = Service(executable_path=r'/usr/bin/chromedriver')
  #return webdriver.Chrome(service=service,options=chrome_options)

In [25]:
all_elements_script = """
function getXPath(element) {
  if (element && element.nodeType === Node.ELEMENT_NODE) {
    const idx = Array.from(element.parentNode.children).indexOf(element) + 1;
    const tag = element.tagName.toLowerCase();
    const parentXPath = getXPath(element.parentNode);
    return `${parentXPath}/${tag}[${idx}]`;
  }
  return '';
}

// Get all elements in the HTML page
const elements = document.getElementsByTagName('*');

// Create an array to store the element details
const elementDetails = [];

// Iterate through each element
for (let i = 0; i < elements.length; i++) {
  const element = elements[i];

  // Get the bounding rectangle of the element
  const rect = element.getBoundingClientRect();

  // Get the text content of the element
  const textContent = element.textContent;

  // Get the tooltip value if it exists
  const tooltip = element.hasAttribute('title') ? element.getAttribute('title') : '';

  // Get the aria-label value
  const ariaLabel = (element.hasAttribute('aria-label') ? element.getAttribute('aria-label') : '');

  // Get the nodeName
  const e_type = element.nodeName;

  // Get the data-initial-value
  const data_initial_value = (element.hasAttribute('data-initial-value') ? element.getAttribute('data-initial-value') : '')

  // Get innerText
  const innerText = element.innerText

  // Store the element, its bounding rectangle, text content, and tooltip details
  const elementInfo = {
    element: element,
    rect: rect,
    textContent: textContent !== undefined ? textContent.trim().replaceAll(",",";"): "",
    ariaLabel: ariaLabel.replaceAll(",",";"),
    tooltip: tooltip.replaceAll(",",";"),
    e_type: e_type  !== undefined ? e_type.replaceAll(",",";"): "",
    data_initial_value: data_initial_value.replaceAll(",",";"),
    innerText: innerText !== undefined ? innerText.replaceAll(",",";"): "",
    parent_xpath : getXPath(element.parentElement)
  };

  if (elementInfo.rect !== undefined){
  elementDetails.push(elementInfo)
  }

}

// Display the element details
console.log("centerX,centerY,ElementType,textContent,TooltipValue,AriaLabel,data-initial-value,innerText,parent_xpath,height,width,top,bottom");
console.log(elementDetails.map( e=> (e.rect.left + (e.rect.width / 2))+","+(e.rect.top + (e.rect.height / 2))+","+e.e_type+","+e.textContent+","+e.tooltip+","+e.ariaLabel+","+e.data_initial_value+","+e.innerText+","+e.parent_xpath+","+e.rect.height+","+e.rect.width+","+e.rect.top+","+e.rect.bottom).join("\\n"));
"""


window_script = """
  //elementInfo.rect.height > 0 && elementInfo.rect.top >= viewpointscroll && elementInfo.rect.bottom < (viewpointscroll+viewportHeight)

  const viewpointscroll = window.pageYOffset || document.documentElement.scrollTop;
  const viewportHeight = window.innerheight || document.documentElement.clientHeight;
  console.log(viewpointscroll)
  console.log(viewportHeight)  
"""

def run_js_code(wd,sub_script):
  script = f"""
  var consoleLogs = [];
  var originalLog = console.log;
  console.log = function(message) {{
      consoleLogs.push(message);
      originalLog.apply(console, arguments);
  }};

  {sub_script}

  return consoleLogs;
  """
  return wd.execute_script(script)

def screen_to_window_dim(wd):
    logs = run_js_code(wd,window_script)
    assert len(logs) == 2
    viewpointscroll = int(logs[0])
    viewportHeight = int(logs[1])

    return viewpointscroll,viewportHeight

def screen_to_table(wd):
  import pandas as pd
  import io

  logs = run_js_code(wd,all_elements_script)
  try:
     return pd.read_csv(io.StringIO("\n".join(logs)), sep=",")
  except Exception as e:
    print(f"WARNING:\n On Table Data: {logs}\n Error {e}")
    return pd.read_csv(io.StringIO("\n".join(logs)), sep=",",on_bad_lines="skip")

In [26]:
import pandas as pd
import re

def is_css_code(text):
    pattern = r"\{.*?\}"

    # Find all matches of CSS rules in the text
    matches = re.findall(pattern, text)

    # Print the matched CSS rules
    for match in matches:
        return True
    return False

def drop_with_exists_finer_element(df):
    df.parent_xpath = df.parent_xpath.fillna("")
    indexs_to_drop = list()
    for index,row in df.iterrows():
        sub_elements = df[df.parent_xpath.str.startswith(f"{row['parent_xpath']}/")]
        drop_row = any(sub_elements.textContent.apply(lambda x: x in row.textContent))
        if drop_row:
            indexs_to_drop.append(index)
    return df.drop(index=indexs_to_drop)

def remove_elements_without_size(df):
     _df =  df[(~df.width.isna()) & (~df.height.isna())]
     _df.loc[:,"width"] = pd.to_numeric(_df.loc[:,"width"], errors='coerce').fillna(0.0).astype(float)
     _df.loc[:,"height"]  = pd.to_numeric(_df.loc[:,"height"], errors='coerce').fillna(0.0).astype(float)
     return _df[(_df.width > 0) & (_df.height > 0)]

def remove_out_of_window(df,viewpointscroll,viewportHeight):
    #height,width,top,bottom
    df.bottom = pd.to_numeric(df.bottom, errors='coerce').fillna(0.0).astype(float)
    df.top = pd.to_numeric(df.top, errors='coerce').fillna(0.0).astype(float)
    return df[(df.top >= viewpointscroll) & (df.bottom < (viewpointscroll+viewportHeight))]


def remove_without_textual_infomration(df):  
    return df[(df.innerText != '') | (df['data-initial-value'] != '') | (df.TooltipValue != '') | (df.textContent != '') | (df.AriaLabel != '')]

def minimize_page_data(df,viewpointscroll,viewportHeight):
    
    _df = df.copy()
    _df = remove_elements_without_size(_df)
    _df = remove_out_of_window(_df,viewpointscroll,viewportHeight)
    _df = remove_without_textual_infomration(_df)


    # if all the information is the same, take the deeper element (which is the last becuase of our listing method)
    _df.drop_duplicates(subset=['textContent','TooltipValue','AriaLabel','data-initial-value'],keep='last',inplace=True)

    
    # remove JS comments
    _df.textContent = _df.textContent.apply(lambda x:str(x).split("//#")[0])
    # remove elements contains css code
    _df =  _df.loc[(_df['textContent'].apply(is_css_code) == False) | _df.textContent.isna()]

    _df =  drop_with_exists_finer_element(_df)

    _df.drop(columns=['parent_xpath','height','width','top','bottom'],inplace=True)
    return _df


def dataframe_diff(df1, df2,viewpointscroll,viewportHeight):
    
    """
    Calculates the difference between two DataFrames.
    Returns two DataFrames: one for removed rows and one for added rows.
    """
    if df1 is None:
        return "There was not page before"
    df1_list = df1.to_csv(index=False).split('\n')[1:]
    df2_list = df2.to_csv(index=False).split('\n')[1:]

    added_to_changed =  pd.DataFrame([row.split(",") for row in df2_list if row not in df1_list],columns=list(df1.columns))
    removed = pd.DataFrame([row.split(",") for row in df1_list if row not in df2_list],columns=list(df1.columns))

    return f"The Elements that was removed or changed are:\n {minimize_page_data(removed,viewpointscroll,viewportHeight).to_csv(index=False)} "+ \
             f"The Elements that was added or changed are:\n {minimize_page_data(added_to_changed,viewpointscroll,viewportHeight).to_csv(index=False)} "

In [27]:
web_driver = start_browesr()
web_driver.get("https://www.google.com")
df = screen_to_table(web_driver)
viewpointscroll,viewportHeight = screen_to_window_dim(web_driver)
df = minimize_page_data(df,viewpointscroll,viewportHeight)
web_driver.quit()
df.head()

,centerX,centerY,ElementType,textContent,TooltipValue,AriaLabel,data-initial-value,innerText
487,291.0703125,30,A,Gmail,NaN,Gmail (פתיחה בכרטיסייה חדשה),NaN,Gmail
489,222.046875,30,A,חיפוש תמונות,NaN,חיפוש תמונות (פתיחה בכרטיסייה חדשה),NaN,חיפוש תמונות
494,146,30,A,nan,NaN,אפליקציות Google,NaN,NaN
498,66,30,SPAN,כניסה,NaN,NaN,NaN,כניסה
509,521,250,IMG,nan,122 שנים להולדת מוחמד חלמי,NaN,NaN,NaN


In [28]:
def web_driver_to_image(wd,file_name):
  full_path = f"{file_name}.png"
  wd.save_screenshot(full_path)
  return full_path

def web_driver_to_html(wd,file_name):
  # Get the page source HTML
  html_content = wd.page_source
  full_path = f"{file_name}.html"
  # Save the HTML content to a file
  with open(full_path, 'w', encoding='utf-8') as f:
      f.write(html_content)
  return full_path


import os
import base64
from PIL import Image
from IPython.display import display, HTML

def display_images_side_by_side(before_file, after_file):
    if not os.path.isfile(before_file):
        print(f"Error: File '{before_file}' not found.")
        return
    if not os.path.isfile(after_file):
        print(f"Error: File '{after_file}' not found.")
        return

    _, before_ext = os.path.splitext(before_file)
    _, after_ext = os.path.splitext(after_file)
    valid_extensions = ['.png', '.jpg', '.jpeg', '.gif']

    if before_ext.lower() not in valid_extensions:
        print(f"Error: Invalid file type. Only {', '.join(valid_extensions)} are supported.")
        return
    if after_ext.lower() not in valid_extensions:
        print(f"Error: Invalid file type. Only {', '.join(valid_extensions)} are supported.")
        return

    try:
        display_side_by_side([
            (before_file, 'Before'),
            (after_file, 'After')
        ])
    except Exception as e:
        print(f"Error displaying images: {e}")

def display_side_by_side(images_with_titles):
    html = "<style>td img{max-width:100%; max-height:100%;} td.title-cell{text-align:center; font-size:18px;}</style>"
    html += "<table>"
    html += "<tr>"
    for _, title in images_with_titles:
        html += f"<td class='title-cell'>{title}</td>"
    html += "</tr>"
    html += "<tr>"
    for file, _ in images_with_titles:
        image_data = base64.b64encode(open(file, 'rb').read()).decode('utf-8')
        html += f"<td><img src='data:image/png;base64,{image_data}' /></td>"
    html += "</tr>"
    html += "</table>"

    display(HTML(html))

def draw_on_screen(webdriver,filename,x,y,**kwarg):
  from PIL import Image, ImageDraw,ImageFont
  # Perform mouse click at X and Y coordinates
  # Open the screenshot image using Pillow
  final_fname = f"{filename}_click_location"
  final_fname = web_driver_to_image(webdriver,final_fname)
  image = Image.open(final_fname)

  # Create a drawing context on the image
  draw = ImageDraw.Draw(image)

  # Define the size of the marker
  marker_size = 10

  # Draw a marker at the specified coordinates
  draw.rectangle([(x - marker_size, y - marker_size), (x + marker_size, y + marker_size)], outline="red")

  if "text" in kwarg:
    text_x = x  # X coordinate for the text (centered with the rectangle)
    text_y = y - marker_size - 20  # Y coordinate above the rectangle
    draw.text((text_x, text_y), kwarg['text'], fill="black")


  # Save the marked screenshot

  image.save(final_fname)
  return filename

def get_screen_size(webdriver):
  window_size = webdriver.get_window_size()
  width = window_size["width"]
  height = window_size["height"]
  return f"width={width},height={height}"


def get_scroll_height(web_driver):
    import time
    initial_scroll_position = web_driver.execute_script("return window.pageYOffset")

    # Scroll down a bit
    web_driver.execute_script("window.scrollBy(0, 100);")

    # Wait for a brief moment
    time.sleep(1)

    # Get the scroll position after scrolling down
    scroll_down_position = web_driver.execute_script("return window.pageYOffset")

    # Scroll up to the initial position
    web_driver.execute_script(f"window.scrollTo(0, {initial_scroll_position});")

    # Wait for a brief moment
    time.sleep(1)

    # Scroll up a bit
    web_driver.execute_script("window.scrollBy(0, -100);")

    # Wait for a brief moment
    time.sleep(1)

    # Get the scroll position after scrolling up
    scroll_up_position = web_driver.execute_script("return window.pageYOffset")

    # Scroll back to the initial position
    web_driver.execute_script(f"window.scrollTo(0, {initial_scroll_position});")

    # Compare the scroll positions
    if scroll_down_position > initial_scroll_position or scroll_up_position < initial_scroll_position:
        return "Client can scroll both up and down!"
    elif scroll_down_position > initial_scroll_position:
        return "Client can scroll down!"
    elif scroll_up_position < initial_scroll_position:
        return "Client can scroll up!"
    else:
        return "Client cannot scroll either up or down!"

def get_scroll_width(web_driver):
    import time
    initial_scroll_position = web_driver.execute_script("return window.pageXOffset")

    # Scroll right a bit
    web_driver.execute_script("window.scrollBy(100, 0);")

    # Wait for a brief moment
    time.sleep(1)

    # Get the scroll position after scrolling right
    scroll_right_position = web_driver.execute_script("return window.pageXOffset")

    # Scroll left to the initial position
    web_driver.execute_script(f"window.scrollTo({initial_scroll_position}, 0);")

    # Wait for a brief moment
    time.sleep(1)

    # Scroll left a bit
    web_driver.execute_script("window.scrollBy(-100, 0);")

    # Wait for a brief moment
    time.sleep(1)

    # Get the scroll position after scrolling left
    scroll_left_position = web_driver.execute_script("return window.pageXOffset")

    # Scroll back to the initial position
    web_driver.execute_script(f"window.scrollTo({initial_scroll_position}, 0);")

    # Compare the scroll positions
    if scroll_right_position > initial_scroll_position or scroll_left_position < initial_scroll_position:
        return "Client can scroll both left and right!"
    elif scroll_right_position > initial_scroll_position:
        return "Client can scroll right!"
    elif scroll_left_position < initial_scroll_position:
        return "Client can scroll left!"
    else:
        return "Client cannot scroll either left or right!"


def get_scroll_options(web_driver):
    width = get_scroll_width(web_driver)
    height = get_scroll_height(web_driver)
    return f"On the Width Axis, {width}. On the Height Axis, {height}"

def to_text_file(text,filename):
    with open(filename, 'a') as file:
        file.write(text)

# Tools
base on https://github.com/mpaepper/llm_agents/blob/main/llm_agents/tools/google_search.py

In [29]:
from pydantic import BaseModel

class ToolInterface(BaseModel):
    name: str
    description: str
    click_on_screen:bool = False

    def is_click_on_screen(self) -> bool:
      return self.click_on_screen



def example_tool(tool,setup_function=None,*arg,**kwarg):
  wd = start_browesr()
  if setup_function:
    setup_function(wd)
  b_filename = web_driver_to_image(wd,f"{str(tool.__class__)}_before")
  tool().use(wd,*arg,**kwarg)
  a_filename = web_driver_to_image(wd,f"{str(tool.__class__)}_after")
  display_images_side_by_side(b_filename,a_filename)

In [30]:
def click_on_screen(wd, x, y):
  js_script = f"return document.elementFromPoint({x}, {y})"
  input_field = wd.execute_script(js_script)
  # Enter the text into the input field
  input_field.click()
  return wd


class ClickOnCoordinates(ToolInterface):
  """Click on certain coordinate on the screen """

  name :str = "Click on coordinates on the screen"
  description:str = "click on x,y coordinates in order to move to the next screen. Input format: {{\"x\": <place_num_here>,\"y\":<place_num_here>}}"
  click_on_screen:bool = True

  def use(self,web_driver:object, x: float, y:float) -> str:
      click_on_screen(web_driver,x,y)

In [31]:

class GoToURL(ToolInterface):
  """ Go to a specific url address """

  name:str = "Go to a specific url web address"
  description:str = "Change the url to a provied URL. Input format: {{\"url\":\"<place_url_here>\"}}"
  click_on_screen:str = True

  def use(self, web_driver:object, url: str)-> None:
      web_driver.get(url)


def change_url(web_driver,first_page):
  web_driver.get(first_page)

#example_tool(GoToURL,url="https://www.google.com/",setup_function=lambda wd:change_url(wd,"https://www.n12.co.il/"))

In [32]:
class ScrollDown(ToolInterface):
    """Scroll down the web page by half the screen height"""

    name:str = "Scroll Down"
    description:str = "Scroll down the web page by half the screen height, no input."

    def use(self, web_driver: object) -> None:
        # Get the height of the web page
        page_height = web_driver.execute_script("return document.body.scrollHeight")

        # Get the height of the viewport
        viewport_height = web_driver.execute_script("return window.innerHeight")

        # Calculate the scroll distance (half the screen height)
        scroll_distance = viewport_height // 2

        # Scroll down the web page
        web_driver.execute_script(f"window.scrollBy(0, {scroll_distance});")


#example_tool(ScrollDown,setup_function=lambda wd:change_url(wd,"https://www.n12.co.il/"))

In [33]:
class ScrollUp(ToolInterface):
    """Scroll up the web page by half the screen height"""

    name:str = "Scroll Up"
    description:str = "Scroll up the web page by half the screen height, no input."

    def use(self, web_driver: object) -> None:
        # Get the height of the viewport
        viewport_height = web_driver.execute_script("return window.innerHeight")

        # Calculate the scroll distance (half the screen height)
        scroll_distance = viewport_height // 2

        # Scroll up the web page
        web_driver.execute_script(f"window.scrollBy(0, -{scroll_distance});")

#example_tool(ScrollUp,setup_function=lambda wd:change_url(wd,"https://stackoverflow.com/a/20464320"))

In [34]:
class ScrollRight(ToolInterface):
    """Scroll the web page to the right by half the screen width"""

    name :str= "Scroll Right"
    description:str = "Scroll the web page to the right by half the screen width, no input"

    def use(self, web_driver: object) -> None:
        # Get the width of the viewport
        viewport_width = web_driver.execute_script("return window.innerWidth")

        # Calculate the scroll distance (half the screen width)
        scroll_distance = viewport_width // 2

        # Scroll the web page to the right
        web_driver.execute_script(f"window.scrollBy({scroll_distance}, 0);")

#example_tool(ScrollRight,setup_function=lambda wd:change_url(wd,"https://www.n12.co.il/"))

In [35]:
from selenium.webdriver.common.action_chains import ActionChains

class GoBack(ToolInterface):
    """go back to previous page"""

    name:str = "Go Back"
    description:str = "Go back to the previous page,no input."

    def use(self, web_driver: object) -> None:
        # Simulate clicking the browser's "Next" button
        web_driver.back()

def change_url_twice(web_driver,first_page,second_page):
  web_driver.get(first_page)
  web_driver.get(second_page)

#example_tool(GoBack,setup_function=lambda wd:change_url_twice(wd,"https://www.google.com","https://www.bbc.com/"))

In [36]:
from selenium.webdriver.common.action_chains import ActionChains

class Refresh(ToolInterface):
    """go back to previous page"""

    name:str = "Refresh page"
    description:str = "refresh the current page,no input."

    def use(self, web_driver: object) -> None:
        # Simulate clicking the browser's "Next" button
        web_driver.refresh()


In [37]:
class EnterText(ToolInterface):
    """Click on a field and enter text"""

    name:str = "Enter Text"
    description:str = "Click on a field and enter text, Input format: {{\"text\":\"<text_to_enter>\",\"x\": <place_num_here>,\"y\":<place_num_here>}}"
    click_on_screen:bool = True

    def use(self, web_driver: object, x:float ,y:float, text: str) -> None:
        js_script = f"return document.elementFromPoint({x}, {y})"
        input_field = web_driver.execute_script(js_script)
        #print(input_field)
        # Enter the text into the input field
        input_field.click()
        input_field.send_keys(text)
#example_tool(EnterText,x=250,y=250,text="text to enter",setup_function=lambda wd:change_url(wd,"https://docs.google.com/document/d/1o1dTLtEeLGJ9iVAWHolGivi1RYnv2JbcSVEkP7PXB-Q/edit?usp=sharing"))

In [43]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

class HitAKey(ToolInterface):
    """Click on a field and enter text"""

    name:str = "Hit A Key"
    description:str = "Hit on of the keys,  Input format: {{\"text\":\"esc\"}} or {{\"text\":\"enter\"}}"
    click_on_screen:bool = True

    def use(self, web_driver: object, text: str) -> None:
        if text.lower() == "esc":
            webdriver.ActionChains(web_driver).send_keys(Keys.ESCAPE).perform()

        elif text.lower() == "enter":
            webdriver.ActionChains(web_driver).send_keys(Keys.ESCAPE).perform()

#example_tool(EnterText,x=250,y=250,text="text to enter",setup_function=lambda wd:change_url(wd,"https://docs.google.com/document/d/1o1dTLtEeLGJ9iVAWHolGivi1RYnv2JbcSVEkP7PXB-Q/edit?usp=sharing"))

# AutoAgent



In [38]:
import openai


from pydantic import BaseModel
from typing import List


class ChatLLM(BaseModel):
    model: str = 'gpt-3.5-turbo'
    temperature: float = 0
    openai.api_key = os.getenv("OPEN-AI-API") 

    def generate(self, prompt: str, stop: List[str] = None):
        response = openai.ChatCompletion.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
            stop=stop
        )
        return response.choices[0].message.content

In [63]:
def get_text_in_cordinates(web_driver,x,y,**kwarg):
        js_script = f"return document.elementFromPoint({x}, {y}).innerText"
        return web_driver.execute_script(js_script)

In [66]:
import datetime
import re

from pydantic import BaseModel
from typing import List, Dict, Tuple


FINAL_ANSWER_TOKEN = "Final Answer:"
OBSERVATION_TOKEN = "Observation:"
THOUGHT_TOKEN = "Thought:"
PROMPT_TEMPLATE = """

Today is {today}, the site i'm looking on is {site_url}.

Here is a representation of what is see on my screen in a table shape:
{on_screen_data}
Current screen size: 
{screen_size}
Scroll Options: 
{scroll_ratio}
You should accomplish the task given to you as best as you can using the following tools:

{tool_description}

Use the following format:

Question: the input question you must answer.
Thought: comment on what you want to do next.
Input Field: where there is input fields on screen? what are its coordinates? what there are used for? what values they contains? 
Buttons: where there is buttons on screen? what there are used for?
Status: looking at your previous response, what is successful?
Action: the action to take, exactly one element of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation repeats N times, use it until you are sure of the answer)
Thought: I now know the final answer
Final Answer: your final answer to the original input question 

Begin!

Task To Accomplish: {task_to_accomplish}
Previous executions:
{previous_responses}
"""

import time
import shutil
class Agent(BaseModel):
    llm: ChatLLM
    web_driver : object = None
    tools: List[ToolInterface]
    prompt_template: str = PROMPT_TEMPLATE
    max_loops: int = 1
    # The stop pattern is used, so the LLM does not hallucinate until the end
    stop_pattern: List[str] = [f'\n{OBSERVATION_TOKEN}', f'\n\t{OBSERVATION_TOKEN}']

    @property
    def tool_description(self) -> str:
        return "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])

    @property
    def tool_names(self) -> str:
        return ",".join([tool.name for tool in self.tools])

    @property
    def tool_by_names(self) -> Dict[str, ToolInterface]:
        return {tool.name: tool for tool in self.tools}

    def run(self, task_to_accomplish: str, url:str):
        shutil.rmtree('outputs',ignore_errors=True)
        os.mkdir("outputs")
        self.web_driver.set_window_size(1024, 768)
        on_screen = None
        try:
            self.web_driver.get(url)
            previous_responses = []
            previous_responses_status = ""
            num_loops = 0
     
            # compute the elements on screen, current + change
            on_screen = screen_to_table(self.web_driver)
            on_screen.to_csv("outputs/step_1.csv")
            viewpointscroll,viewportHeight = screen_to_window_dim(self.web_driver)
            
            on_screen_diff = dataframe_diff(None,on_screen,viewpointscroll,viewportHeight)

            # get screen size
            screen_size = get_screen_size(self.web_driver)
            file_name_png = web_driver_to_image(self.web_driver,"outputs/step_1")
            file_name_html = web_driver_to_html(self.web_driver,"outputs/step_1")
            scroll_ratio = get_scroll_options(self.web_driver)
            
            prompt = self.prompt_template.format(
                    today = datetime.date.today(),
                    tool_description=self.tool_description,
                  tool_names=self.tool_names,
                  task_to_accomplish=task_to_accomplish,
                  on_screen_data="{on_screen_data}",
                  previous_responses='{previous_responses}',
                  screen_size="{screen_size}",
                  scroll_ratio="{scroll_ratio}",
                  on_screen_diff="{on_screen_diff}",
                  site_url=url
            )

            print(f"StaticPrompt={prompt}")
            while num_loops < self.max_loops:
                num_loops += 1
                print(f"--- Iteration {num_loops} ---")

                curr_prompt = prompt.format(previous_responses="\n".join(previous_responses),
                                            on_screen_data=minimize_page_data(on_screen,viewpointscroll,viewportHeight).to_csv(index=False),
                                            screen_size=screen_size,scroll_ratio=scroll_ratio,
                                            on_screen_diff=on_screen_diff,
                )
                

                print("\n\n#input:")
                print(f"screenshot={file_name_png}")
                print(f"html={file_name_html}")
                print(f"Prompt={curr_prompt}")
                
                try:
                    to_text_file(curr_prompt,f"outputs/step_{str(num_loops)}_prompt.txt")
                    generated, tool, tool_input = self.decide_next_action(curr_prompt)
                    to_text_file(generated,f"outputs/step_{str(num_loops)}_response.txt")

                    if tool == 'Final Answer':
                        return tool_input
                
                    if tool not in self.tool_by_names:
                        raise ValueError(f"unknown tool:{tool}")
            
                    tool_executor = self.tool_by_names[tool]
                    initial_page_url = self.web_driver.current_url

                    if tool_executor.is_click_on_screen():
                        draw_on_screen(self.web_driver,f"outputs/step_{str(num_loops)}",**tool_input)
                    try:
                        tool_executor.use(self.web_driver,**tool_input)
                        # 
                        # if tool_executor.is_click_on_screen():
                        #     print(get_text_in_cordinates(self.web_driver,**tool_input))
                    except Exception:
                        raise ValueError("tool execution failed.")
                    
                    # if the tool worked, wait and sample the site again. 
                    if initial_page_url != self.web_driver.current_url:
                        time.sleep(10)

                    # compute the elements on screen, current + change
                    temp_on_screen = screen_to_table(self.web_driver)
                    temp_on_screen.to_csv(f"outputs/step_{str(num_loops+1)}.csv")
                    viewpointscroll,viewportHeight = screen_to_window_dim(self.web_driver)
                    
                    on_screen_diff = dataframe_diff(on_screen,temp_on_screen,viewpointscroll,viewportHeight)
                    on_screen = temp_on_screen

                    screen_size = get_screen_size(self.web_driver)
                    scroll_ratio = get_scroll_options(self.web_driver)
                    file_name_png = web_driver_to_image(self.web_driver,f"outputs/step_{str(num_loops+1)}")
                    file_name_html = web_driver_to_html(self.web_driver,f"outputs/step_{str(num_loops+1)}")
                    
                    previous_responses_status="successful."
                except ValueError as e:
                    previous_responses_status = f"failed, {str(e)}"

                
                #\n{OBSERVATION_TOKEN} \n{THOUGHT_TOKEN}\n
                previous_responses.append(f"Previous {num_loops} response:\n{generated}\nexecution status:{previous_responses_status}")

            print("------ Final Screen ------")
            print(f"screenshot={file_name_png}")
        except Exception as e:
            print(f"FAILED: {str(e)}")
        finally:
            try:
                if self.web_driver:
                  self.web_driver.close()
                  self.web_driver.quit()
            finally:
                pass

    def parse_json(self,tool_input:str):
      import json
      try:
        response = json.loads(tool_input)
      except Exception as e:
        raise ValueError(f"the output `{tool_input}` is not a JSON, error = {e}")
      return response

    def decide_next_action(self, prompt: str) -> str:
        print("\n\n#output:")

        
        generated = self.llm.generate(prompt, stop=self.stop_pattern)

        print(f"=Gnerated=\n{generated}\n==========")

        tool, tool_input = self._parse(generated)
        print(f"Tool={tool}")
        print(f"Args={tool_input}")

        return generated, tool, self.parse_json(tool_input)

    
    @staticmethod
    def _parse(generated: str) -> Tuple[str, str]:
        if FINAL_ANSWER_TOKEN in generated:
            return "Final Answer", generated.split(FINAL_ANSWER_TOKEN)[-1].strip()

        if "Action Input" in generated:
            regex = r"Action: [\[]?(.*?)[\]]?[\n]*Action Input:[\s]*(.*)"
            match = re.search(regex, generated, re.DOTALL)
            if not match:
                raise ValueError(f"the output `{generated}` is not matching the expected format.")
            tool = match.group(1).strip()
            tool_input = match.group(2)
        else:
            tool = generated.split("Action:")[-1].strip()
            tool_input = "{}"

        def strip_tool(string:str):
            return re.sub(r'[^a-zA-Z ]', '', string).strip(" ").strip('"')
        
        def strip_args(string:str):
            string_temp = string.strip(" ").strip('"')
            string_temp  = "{"+string.split("{")[-1]
            string_temp = string_temp.split("}")[0] + "}"
            return string_temp
        
        return strip_tool(tool), strip_args(tool_input)

In [47]:
avaliable_tools = [ClickOnCoordinates(),EnterText(),GoBack(),ScrollRight(),ScrollUp(),ScrollDown(),Refresh(),HitAKey()]

In [ ]:
agent = Agent(max_loops=0,llm=ChatLLM(), tools=avaliable_tools ,web_driver = start_browesr())
agent.run("Log into my Gmail account, user name is 'erlichsefi@gmail.com', password is '1234567'","https://www.google.com")

In [ ]:
# agent = Agent(max_loops=5,llm=ChatLLM(), tools=avaliable_tools ,web_driver = start_browesr())
# agent.run("Log into my Gmail account, user name is 'erlichsefi@gmail.com', password is '1234567'","https://www.google.com")

In [52]:
# agent = Agent(max_loops=3,llm=ChatLLM(), tools=avaliable_tools, web_driver = start_browesr())
# agent.run("search the bbc site and extract the headline from the cbc website.","https://www.google.com/")

In [ ]:
# agent = Agent(max_loops=3,llm=ChatLLM(), tools=avaliable_tools ,web_driver = start_browesr())
# agent.run("log in to my account,user name is 'erlichsefi@gmail.com', password is '1234567'","https://www.facebook.com/")

In [67]:
agent = Agent(max_loops=7,llm=ChatLLM(), tools=avaliable_tools ,web_driver = start_browesr())
agent.run("log in to my account,user name is 'erlichsefi@gmail.com', password is '1234567'","https://www.wishingwell.co.il/")

StaticPrompt=

Today is 2023-07-25, the site i'm looking on is https://www.wishingwell.co.il/.

Here is a representation of what is see on my screen in a table shape:
{on_screen_data}
Current screen size: 
{screen_size}
Scroll Options: 
{scroll_ratio}
You should accomplish the task given to you as best as you can using the following tools:

Click on coordinates on the screen: click on x,y coordinates in order to move to the next screen. Input format: {{"x": <place_num_here>,"y":<place_num_here>}}
Enter Text: Click on a field and enter text, Input format: {{"text":"<text_to_enter>","x": <place_num_here>,"y":<place_num_here>}}
Go Back: Go back to the previous page,no input.
Scroll Right: Scroll the web page to the right by half the screen width, no input
Scroll Up: Scroll up the web page by half the screen height, no input.
Scroll Down: Scroll down the web page by half the screen height, no input.
Refresh page: refresh the current page,no input.
Hit A Key: Hit on of the keys,  Input form

KeyboardInterrupt: 